In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import imageio
import os 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Dense, Flatten

In [4]:
import glob
c1 = glob.glob('Data/chest_xray/train/*/*')
c2 = glob.glob('Data/chest_xray/test/*/*')

print('# Train images : {} \n# Test images : {}'.format(len(c1), len(c2)))

# Train images : 5216 
# Test images : 624


## Loading, resizing, classifying images

In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
          
def returnLabel(path):
    imgClass = tf.strings.split(path,'/')[-2]
    return 0 if imgClass == 'NORMAL' else 1

def returnImg(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    imgClass = returnLabel(path)
    return tf.image.resize(img, (256,256)), imgClass

In [6]:
train_path = 'Data/chest_xray/train'
list_ds = tf.data.Dataset.list_files(str(train_path+'/*/*'))
labeled_ds = list_ds.map(returnImg, num_parallel_calls=AUTOTUNE)
labeled_ds

<ParallelMapDataset shapes: ((256, 256, 3), ()), types: (tf.float32, tf.int32)>

## Model


In [40]:
## To store accuracy and loss of the model to plot the graph later on
class AccuracyHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.loss_ = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('accuracy'))
        self.loss_.append(logs.get('loss'))

history = AccuracyHistory()

## To save weights of the model during training
checkpoint_path = 'Checkpoints/cp-{epoch:04d}.ckpt'
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True, save_freq=5)

In [16]:
def nn_model():
    model = Sequential()
    
    model.add(Conv2D(8,5, strides=(5,5), padding='SAME', activation='relu', use_bias=True))
    model.add(AveragePooling2D(pool_size=(2,2), data_format='channels_last'))
    
    model.add(Conv2D(10, 5, activation='relu', use_bias=True))
    model.add(AveragePooling2D(pool_size=(2,2), strides=(1,1), data_format='channels_last'))
    
    model.add(Flatten())
    
    model.add(Dense(120, activation='relu', use_bias=True))
    model.add(Dense(84, activation='relu', use_bias=True))
    model.add(Dense(1, activation='sigmoid', use_bias=True))
    
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07), metrics=['accuracy'])
    
    return model
    

In [31]:
epochs = 100
batch_size = 128
labeled_data = labeled_ds.batch(5300)

In [29]:
def train(model,dataset, batch_size, epochs):
    model.fit(dataset, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[history, cp_callback])
    return model
model = train(labeled_data, batch_size, epochs)

Epoch 1/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6602 - accuracy: 0.7487
Epoch 2/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6027 - accuracy: 0.7429
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 0.5747 - accuracy: 0.7429
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 0.5530 - accuracy: 0.7429
Epoch 5/100

Epoch 00005: saving model to Checkpoints/cp-0005.ckpt
1/1 [==============================] - 0s 1ms/step - loss: 0.5415 - accuracy: 0.7429
Epoch 6/100
1/1 [==============================] - 0s 1ms/step - loss: 0.5251 - accuracy: 0.7429
Epoch 7/100
1/1 [==============================] - 0s 1ms/step - loss: 0.5034 - accuracy: 0.7429
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 0.4742 - accuracy: 0.7429
Epoch 9/100
1/1 [==============================] - 0s 1ms/step - loss: 0.4442 - accuracy: 0.7450
Epoch 10/100

Epoch 00010: saving model to Checkpoints/cp-0010.ckpt
1/1 

1/1 [==============================] - 0s 1ms/step - loss: 0.1124 - accuracy: 0.9571
Epoch 77/100
1/1 [==============================] - 0s 1ms/step - loss: 0.1132 - accuracy: 0.9584
Epoch 78/100
1/1 [==============================] - 0s 1ms/step - loss: 0.1130 - accuracy: 0.9553
Epoch 79/100
1/1 [==============================] - 0s 982us/step - loss: 0.1101 - accuracy: 0.9578
Epoch 80/100

Epoch 00080: saving model to Checkpoints/cp-0080.ckpt
1/1 [==============================] - 0s 1ms/step - loss: 0.1119 - accuracy: 0.9590
Epoch 81/100
1/1 [==============================] - 0s 941us/step - loss: 0.1094 - accuracy: 0.9574
Epoch 82/100
1/1 [==============================] - 0s 1ms/step - loss: 0.1092 - accuracy: 0.9578
Epoch 83/100
1/1 [==============================] - 0s 936us/step - loss: 0.1095 - accuracy: 0.9590
Epoch 84/100
1/1 [==============================] - 0s 977us/step - loss: 0.1073 - accuracy: 0.9592
Epoch 85/100

Epoch 00085: saving model to Checkpoints/cp-0085.ckpt


## Testing

In [35]:
def testing(model, weights, test_dataset):    
    model.load_weights(weights)
    return model.evaluate(test_dataset, verbose = 1, batch_size=128, return_dict=True)
    

In [25]:
## Loading test images
test_path = 'Data/chest_xray/test'
test_ds = tf.data.Dataset.list_files(str(train_path+'/*/*'))
test_labeled = test_ds.map(returnImg, num_parallel_calls=AUTOTUNE)
test_labeled = test_labeled.batch(1000)

In [37]:
## Loading latest set of trained weights
latest_weights = tf.train.latest_checkpoint('Checkpoints')
latest_weights
test_model = nn_model()

In [38]:
test_output = testing(test_model, latest_weights, test_labeled)

6/6 [==============================] - 18s 3s/step - loss: 0.0995 - accuracy: 0.9634


In [42]:
## Uncomment this section to plot the graph, model has to be trained from scratch to plot the graph

# plt.plot(range(100), history.loss_, label='Loss')
# plt.plot(range(100), history.acc, label='Accuracy')
# plt.legend()
# plt.show()